<a href="https://colab.research.google.com/github/ktamaral/model_vector_comparison/blob/main/e5_embedding_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script generates query embeddings using the intfloat/multilingual-e5-large-instruct model in PyTorch to simulate how Elasticsearch creates them during semantic search. The query embedding emulates the Elasticsearch built-in inference pipeline by adding a prefix to the query, tokenizing with the XLM-Roberta algorithm, applying average pooling, and L2-normalizing the vector. The result is a 1024-dimensional embedding that can be compared to a vector generated with the Elasticsearch deployed E5 model.

Resources

https://huggingface.co/intfloat/multilingual-e5-large-instruct

https://colab.research.google.com/drive/16HoNx466TRHwkY-7Nsq_nB_rqdGjj4yO#scrollTo=aiTvMzIlkMb9

https://www.elastic.co/search-labs/blog/multilingual-vector-search-e5-embedding-model

https://huggingface.co/intfloat/multilingual-e5-base#faq

https://discuss.elastic.co/t/text-embedding-different-in-elastic-search-and-python-library/329164/5

https://www.elastic.co/search-labs/blog/how-to-deploy-nlp-text-embeddings-and-vector-search

https://huggingface.co/sentence-transformers/msmarco-MiniLM-L12-v3


In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

model_name = "intfloat/multilingual-e5-large-instruct"

task_description = "Given a web search query, retrieve relevant passages that answer the query"
query_text = "Radio, TV, and Film Broadcasting History"

# Generate the query based on E5 prompt style
# https://huggingface.co/intfloat/multilingual-e5-base#faq
def format_e5_query(task: str, query: str) -> str:
    return f"Instruct: {task}\nQuery: {query}"

formatted_query = format_e5_query(task_description, query_text)

# Use Hugging Face tokenizer and model (matches what Elasticsearch uses)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
# Set to eval mode
model.eval()

# Move model to GPU if available
# https://medium.com/biased-algorithms/mastering-pytorch-to-device-an-advanced-guide-for-efficient-device-management-0290b086f17e
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenization matching Elasticsearch inference config
# https://huggingface.co/transformers/v3.5.1/main_classes/tokenizer.html
inputs = tokenizer(
    formatted_query,
    return_tensors="pt",
    max_length=512,
    # Truncation from the beginning (matches the "first" value)
    truncation=True,
    padding=True
).to(device)

# Average pooling
def mean_pooling(model_output, attention_mask):
    """
    Averages token embeddings to create a single sentence embedding
    https://medium.com/better-programming/implementing-nearest-neighbour-search-with-elasticsearch-c59a8d33dd9d
    """
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, dim=1) / input_mask_expanded.sum(dim=1)

# Run inference and pooling with normalization
# Disable gradient descent
with torch.no_grad():
    outputs = model(**inputs)
    pooled = mean_pooling(outputs, inputs["attention_mask"])
    # L2 normalization
    normalized = F.normalize(pooled, p=2, dim=1)

# Convert to Python list (to match Elasticsearch expected data structure)
embedding_vector = normalized[0].cpu().tolist()

# Print the vector (1024 floats)
print(embedding_vector)


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

[0.018400318920612335, 0.02872045710682869, -0.034044984728097916, -0.007210986223071814, 0.047423120588064194, -0.008325014263391495, -0.0225231796503067, 0.06421448290348053, 0.038214247673749924, -0.016716066747903824, 0.02335277758538723, 0.006237414199858904, -0.043703045696020126, -0.02617163583636284, -0.008301913738250732, -0.018825771287083626, -0.029049744829535484, 0.004623073153197765, 0.0058710332959890366, -0.020840493962168694, 0.036840733140707016, -0.016418319195508957, -0.03851455822587013, -0.03167654201388359, -0.0053256480023264885, -0.0031490700785070658, -0.025675876066088676, -0.043069027364254, -0.036201171576976776, -0.042033661156892776, -0.0335790291428566, 0.013592716306447983, -0.006372254341840744, -0.030792878940701485, -0.04525774344801903, 0.030519604682922363, 0.0006699772202409804, 0.050686657428741455, -0.09527279436588287, 0.022971903905272484, -0.030721984803676605, 0.029994575306773186, 2.3761660486343317e-05, -0.02062700130045414, -0.02293287590

The pytorch model configuration in the script above emulateas the model configuration in Elasticsearch shown after running a GET request on the `_ml/trained_models` API endpoint.

In [ ]:
# GET _ml/trained_models/multilingual-e5-large-instruct

{
  "count": 1,
  "trained_model_configs": [
    {
      "model_id": "multilingual-e5-large-instruct",
      "model_type": "pytorch",
      "created_by": "api_user",
      "version": "12.0.0",
      "create_time": 1741294919938,
      "model_size_bytes": 0,
      "estimated_operations": 0,
      "license_level": "platinum",
      "description": "Model intfloat/multilingual-e5-large-instruct for task type 'text_embedding'",
      "tags": [],
      "metadata": {
        "per_allocation_memory_bytes": 1676042564,
        "per_deployment_memory_bytes": 2235371552
      },
      "input": {
        "field_names": [
          "text_field"
        ]
      },
      "inference_config": {
        "text_embedding": {
          "vocabulary": {
            "index": ".ml-inference-native-000002"
          },
          "tokenization": {
            "xlm_roberta": {
              "do_lower_case": false,
              "with_special_tokens": true,
              "max_sequence_length": 512,
              "truncate": "first",
              "span": -1
            }
          },
          "embedding_size": 1024
        }
      },
      "location": {
        "index": {
          "name": ".ml-inference-native-000002"
        }
      }
    }
  ]
}

# Compare embedding with one retrieved from ES

In [3]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

# Get a vector from ES
es_vector = [
        0.018400337547063828,
        0.028720546513795853,
        -0.0340450257062912,
        -0.007210760377347469,
        0.047423116862773895,
        -0.008325066417455673,
        -0.022523153573274612,
        0.06421443074941635,
        0.0382143035531044,
        -0.016716092824935913,
        0.023352768272161484,
        0.00623743049800396,
        -0.04370304197072983,
        -0.02617158181965351,
        -0.008301926776766777,
        -0.018825799226760864,
        -0.029049741104245186,
        0.004622986540198326,
        0.0058710104785859585,
        -0.02084042876958847,
        0.03684071823954582,
        -0.01641825959086418,
        -0.03851459175348282,
        -0.03167649358510971,
        -0.005325626581907272,
        -0.003148989286273718,
        -0.025675857439637184,
        -0.04306899011135101,
        -0.03620119392871857,
        -0.042033784091472626,
        -0.03357906639575958,
        0.013592674396932125,
        -0.006372207310050726,
        -0.030792856588959694,
        -0.045257795602083206,
        0.030519668012857437,
        0.0006699321675114334,
        0.050686635076999664,
        -0.09527276456356049,
        0.022971931844949722,
        -0.030721882358193398,
        0.029994554817676544,
        0.000023792450519977137,
        -0.020626980811357498,
        -0.022932851687073708,
        0.033887382596731186,
        0.037065308541059494,
        0.001576274516992271,
        -0.007574158255010843,
        0.06224057823419571,
        0.005427408963441849,
        0.03306630998849869,
        -0.040846411138772964,
        -0.023911671712994576,
        -0.03026973269879818,
        0.02908191829919815,
        -0.047235775738954544,
        0.0015943240141496062,
        -0.04450061172246933,
        0.008549126796424389,
        -0.02189592644572258,
        0.008849486708641052,
        0.03219214081764221,
        -0.01579517312347889,
        -0.0026009995490312576,
        0.03507889062166214,
        0.03067263588309288,
        0.01260490994900465,
        -0.02811671607196331,
        0.03686171770095825,
        -0.032229725271463394,
        0.0027354592457413673,
        -0.004308811388909817,
        -0.008402576670050621,
        -0.038150057196617126,
        -0.02620423398911953,
        0.033859215676784515,
        -0.009354153648018837,
        0.06588373333215714,
        -0.0204844381660223,
        0.06793927401304245,
        0.0002626373607199639,
        0.04037829861044884,
        -0.018001217395067215,
        0.040368858724832535,
        0.030369389802217484,
        0.02166805788874626,
        0.026253314688801765,
        0.03596216067671776,
        0.017163118347525597,
        -0.01796829141676426,
        0.03939425200223923,
        0.013687294907867908,
        0.013651484623551369,
        -0.012785907834768295,
        -0.025295620784163475,
        0.0065466491505503654,
        0.007922457531094551,
        -0.005925590638071299,
        -0.04516265168786049,
        -0.03465859964489937,
        -0.037863314151763916,
        0.012768960557878017,
        -0.007574136834591627,
        -0.012462824583053589,
        0.00899689830839634,
        0.033339183777570724,
        0.01146701443940401,
        -0.010371149517595768,
        0.018158867955207825,
        0.0166721660643816,
        0.05326496809720993,
        -0.005995301995426416,
        -0.006865397561341524,
        -0.05324177071452141,
        0.02822832204401493,
        0.02506551519036293,
        -0.019274374470114708,
        0.002378703560680151,
        0.02324172481894493,
        0.03629819676280022,
        0.01263797003775835,
        0.02127070352435112,
        -0.03966333717107773,
        0.05568716302514076,
        -0.026330238208174706,
        0.023742998018860817,
        0.046597670763731,
        -0.009215793572366238,
        0.0005767084658145905,
        0.022169647738337517,
        0.01815890520811081,
        -0.02277766354382038,
        -0.04415697976946831,
        -0.04603046551346779,
        -0.02606351114809513,
        -0.0542214997112751,
        -0.01235421746969223,
        0.02035357989370823,
        0.0039317007176578045,
        0.0132145369425416,
        0.015322907827794552,
        0.026089061051607132,
        -0.011536262929439545,
        -0.046336252242326736,
        -0.040206968784332275,
        -0.05761444568634033,
        0.029616432264447212,
        -0.03795510157942772,
        0.000563783454708755,
        -0.012620693072676659,
        -0.028514772653579712,
        0.0014137852704152465,
        0.006271795369684696,
        0.013529634103178978,
        0.015697816386818886,
        0.012469563633203506,
        0.051260486245155334,
        -0.02081783302128315,
        0.03036143071949482,
        -0.027271632105112076,
        -0.028559057042002678,
        -0.01881861314177513,
        -0.040509410202503204,
        -0.0009560782928019762,
        -0.024473093450069427,
        0.06022931635379791,
        0.004018393345177174,
        0.02617119438946247,
        0.006328499410301447,
        -0.0099595682695508,
        -0.01880803145468235,
        0.0328834243118763,
        -0.03245045617222786,
        -0.041255440562963486,
        0.030881470069289207,
        0.038640860468149185,
        0.03618432581424713,
        0.03348402678966522,
        -0.04655251279473305,
        0.0006980224279686809,
        0.035255953669548035,
        -0.0014386771945282817,
        0.016165578737854958,
        -0.03088959865272045,
        0.004401621874421835,
        0.029881587252020836,
        0.003630208084359765,
        0.01850748248398304,
        0.021655017510056496,
        0.02470732107758522,
        0.019520293921232224,
        -0.02665138617157936,
        0.060892269015312195,
        -0.020410386845469475,
        -0.01619667187333107,
        0.042650699615478516,
        0.017309993505477905,
        -0.041661083698272705,
        -0.04413093999028206,
        -0.032487839460372925,
        -0.0016935508465394378,
        -0.06674092262983322,
        0.030811158940196037,
        0.026940099895000458,
        -0.03843637555837631,
        -0.04228472337126732,
        -0.027855755761265755,
        -0.004396295174956322,
        -0.02280510775744915,
        -0.05364574119448662,
        -0.03207910433411598,
        0.06186550855636597,
        0.012118781916797161,
        -0.0021480664145201445,
        -0.029540251940488815,
        -0.04582888260483742,
        0.010685176588594913,
        0.02317798137664795,
        0.04096901789307594,
        0.029745468869805336,
        0.04154001921415329,
        0.012628735043108463,
        -0.024705003947019577,
        0.0412425622344017,
        -0.0002423037076368928,
        0.02529141679406166,
        -0.004597316961735487,
        0.0209857989102602,
        0.017412707209587097,
        0.02372722327709198,
        0.030823322013020515,
        -0.02131938934326172,
        -0.020563194528222084,
        0.019756434485316277,
        -0.02120446413755417,
        -0.004624294117093086,
        -0.004062849096953869,
        -0.0009747094591148198,
        0.0449979230761528,
        -0.00901003461331129,
        0.01689707115292549,
        -0.04288988932967186,
        -0.00807915162295103,
        -0.03707568347454071,
        -0.0179370678961277,
        0.0012453728122636676,
        -0.015167642384767532,
        0.01815226301550865,
        -0.04064633697271347,
        0.024586163461208344,
        0.020189404487609863,
        -0.021525274962186813,
        -0.033842358738183975,
        0.03893449157476425,
        0.03177382051944733,
        0.020782073959708214,
        -0.028938734903931618,
        -0.009542908519506454,
        -0.02708299830555916,
        -0.019877301529049873,
        -0.005086922086775303,
        -0.0025833011604845524,
        0.042781051248311996,
        0.028546299785375595,
        -0.006168010178953409,
        -0.01630447991192341,
        -0.02294500172138214,
        0.012345753610134125,
        -0.03333282843232155,
        -0.0474381148815155,
        -0.011014365591108799,
        -0.019175566732883453,
        0.0031406350899487734,
        -0.03848130628466606,
        0.002048858441412449,
        0.0006813225918449461,
        -0.02874695137143135,
        -0.04126102477312088,
        -0.039857201278209686,
        0.014242974109947681,
        -0.028845027089118958,
        0.019259676337242126,
        -0.032811086624860764,
        -0.016809994354844093,
        -0.06131114065647125,
        0.05545441806316376,
        -0.007699844427406788,
        0.030804645270109177,
        -0.05085688456892967,
        0.04391702264547348,
        -0.023950954899191856,
        0.0811900943517685,
        0.011819572187960148,
        0.029892422258853912,
        0.04513547196984291,
        -0.051194384694099426,
        0.011890423484146595,
        -0.03480205684900284,
        0.02078767865896225,
        -0.019805453717708588,
        -0.04977652058005333,
        0.025405699387192726,
        0.040983058512210846,
        -0.014278357848525047,
        0.0003860009310301393,
        -0.019174277782440186,
        0.034161556512117386,
        -0.04420143738389015,
        -0.0017530391924083233,
        -0.012478282675147057,
        0.004940893966704607,
        -0.01638110540807247,
        -0.030507784336805344,
        -0.040385521948337555,
        -0.003222518367692828,
        -0.022234298288822174,
        0.024258596822619438,
        -0.012110142037272453,
        0.04512978345155716,
        -0.0019400706514716148,
        -0.0012184298830106854,
        -0.03959818184375763,
        0.03767944499850273,
        0.023815294727683067,
        -0.014224203303456306,
        0.018825579434633255,
        -0.006170612759888172,
        0.007759897969663143,
        0.0027435175143182278,
        0.004457956179976463,
        -0.019361533224582672,
        -0.010806885547935963,
        0.06326507031917572,
        -0.02738184668123722,
        0.002282540313899517,
        -0.029036397114396095,
        0.0021592192351818085,
        0.024166004732251167,
        -0.009742612019181252,
        0.021609734743833542,
        -0.027980301529169083,
        -0.050509680062532425,
        -0.002436548238620162,
        0.017840780317783356,
        -0.031146613880991936,
        -0.008788608014583588,
        -0.051656320691108704,
        0.03598042204976082,
        0.0070541780441999435,
        0.03238624706864357,
        -0.016554035246372223,
        0.029077408835291862,
        0.029721485450863838,
        0.0011561779538169503,
        0.06979728490114212,
        0.009244518354535103,
        -0.038003407418727875,
        0.04833335056900978,
        -0.012695410288870335,
        0.018559688702225685,
        0.021594766527414322,
        0.05279001221060753,
        -0.03364674746990204,
        -0.03616749495267868,
        -0.011902992613613605,
        0.03280850499868393,
        -0.03779047727584839,
        -0.006266414187848568,
        -0.017930882051587105,
        0.05036051198840141,
        0.009713849052786827,
        -0.026695558801293373,
        0.02819054014980793,
        0.035256192088127136,
        -0.04018380865454674,
        -0.011102549731731415,
        -0.0409291572868824,
        -0.003994666505604982,
        -0.0135049382224679,
        -0.04339931905269623,
        -0.03924436867237091,
        0.016103463247418404,
        -0.019047830253839493,
        -0.022520964965224266,
        0.0030975311528891325,
        0.14356954395771027,
        0.005080783274024725,
        0.03962712362408638,
        -0.024080203846096992,
        0.0015466121258214116,
        -0.008563772775232792,
        0.032585155218839645,
        0.012576844543218613,
        0.00845304410904646,
        0.02131868712604046,
        -0.015709633007645607,
        0.04668121412396431,
        0.015361451543867588,
        -0.05614756420254707,
        0.013889893889427185,
        0.029989518225193024,
        0.01942306198179722,
        0.03555857017636299,
        0.03874646872282028,
        -0.011065978556871414,
        0.04488563910126686,
        0.012529046274721622,
        0.07640016824007034,
        0.03588830679655075,
        0.003774794051423669,
        -0.023372765630483627,
        0.009851236827671528,
        0.02911488525569439,
        0.00009488581417826936,
        0.03957396373152733,
        -0.009348816238343716,
        0.015465539880096912,
        -0.03130635246634483,
        -0.0088383499532938,
        0.025824399664998055,
        0.03830299526453018,
        0.033718887716531754,
        -0.023800410330295563,
        0.05073229968547821,
        -0.006989498157054186,
        -0.018155159428715706,
        -0.06447789072990417,
        -0.008355384692549706,
        0.04898718744516373,
        -0.050570156425237656,
        -0.006985937245190144,
        0.03889618441462517,
        -0.0014491500332951546,
        -0.0043233889155089855,
        0.020565614104270935,
        0.04399640113115311,
        0.06557245552539825,
        -0.004648677073419094,
        -0.058973196893930435,
        -0.025013361126184464,
        -0.04008466377854347,
        -0.017881780862808228,
        0.01006379909813404,
        -0.009311742149293423,
        0.04018852114677429,
        0.05950288474559784,
        0.0031041144393384457,
        0.006898165680468082,
        -0.007524424232542515,
        0.027273666113615036,
        0.02834942564368248,
        0.0022782282903790474,
        -0.05695356801152229,
        -0.0374799519777298,
        -0.027978071942925453,
        -0.030347775667905807,
        -0.035260025411844254,
        -0.023487236350774765,
        0.021081021055579185,
        0.02975832112133503,
        -0.02471003495156765,
        -0.01996910758316517,
        0.05523199960589409,
        -0.017425457015633583,
        -0.030694957822561264,
        0.07394538074731827,
        -0.050324395298957825,
        -0.0321013405919075,
        0.037454646080732346,
        0.03414560481905937,
        -0.026336288079619408,
        0.041066139936447144,
        0.03499598428606987,
        -0.06599452346563339,
        0.04186651483178139,
        0.02379000559449196,
        0.03443368896842003,
        0.04985959827899933,
        0.0579688660800457,
        0.012616956606507301,
        -0.03804830461740494,
        0.039455901831388474,
        0.02013264410197735,
        -0.03624613210558891,
        -0.02400561422109604,
        -0.028430500999093056,
        0.05234246701002121,
        -0.028071774169802666,
        -0.023021692410111427,
        0.00667798426002264,
        0.0028112574946135283,
        0.04577863961458206,
        0.041847310960292816,
        0.023495806381106377,
        0.009923884645104408,
        0.03861269727349281,
        -0.01913687214255333,
        0.02974974736571312,
        -0.012226855382323265,
        0.038054101169109344,
        0.03791094943881035,
        0.02267652377486229,
        -0.03677471727132797,
        0.02447090670466423,
        0.01695777103304863,
        -0.00885749701410532,
        0.01139860413968563,
        0.022648535668849945,
        0.03811519220471382,
        -0.04976585507392883,
        0.04455054923892021,
        -0.03426051139831543,
        -0.005693119950592518,
        0.05242237076163292,
        0.003808050649240613,
        0.011504856869578362,
        -0.010683326981961727,
        -0.020472681149840355,
        0.0028544794768095016,
        0.005789642687886953,
        -0.02533477172255516,
        -0.022336017340421677,
        0.010102749802172184,
        -0.026173070073127747,
        0.03177501633763313,
        0.01182167511433363,
        0.035327136516571045,
        -0.09776881337165833,
        0.017940854653716087,
        -0.01046107430011034,
        0.018316974863409996,
        0.007565689273178577,
        -0.03437817469239235,
        -0.04976712539792061,
        -0.007421887945383787,
        0.011067722924053669,
        -0.04987479746341705,
        0.030905377119779587,
        0.014667815528810024,
        0.02022566646337509,
        0.01600721664726734,
        -0.004841180052608252,
        0.011691197752952576,
        0.04105803743004799,
        -0.06260853260755539,
        -0.02892913855612278,
        -0.0024960667360574007,
        -0.023451607674360275,
        0.06138605996966362,
        0.014456922188401222,
        0.022731536999344826,
        -0.03762208670377731,
        0.007345146965235472,
        0.024708081036806107,
        0.043436143547296524,
        -0.041642867028713226,
        -0.033576298505067825,
        0.0007910797139629722,
        0.021585842594504356,
        0.025975730270147324,
        -0.0362820029258728,
        -0.0261407271027565,
        0.0017024867702275515,
        -0.04408330097794533,
        0.023452945053577423,
        0.02324794977903366,
        0.061899568885564804,
        0.055811215192079544,
        -0.06605993956327438,
        -0.048808448016643524,
        0.036304771900177,
        -0.008953507989645004,
        -0.025824451819062233,
        -0.03880132734775543,
        0.018939975649118423,
        -0.025706494227051735,
        0.06041135638952255,
        -0.026616742834448814,
        0.01907970942556858,
        -0.003633231855928898,
        0.028540052473545074,
        -0.038798220455646515,
        0.00756297679618001,
        -0.02723430097103119,
        -0.05165683105587959,
        -0.003468969836831093,
        -0.00824844092130661,
        -0.03803930804133415,
        -0.01223539188504219,
        -0.03996497765183449,
        -0.030648434534668922,
        -0.005038111004978418,
        0.01363422628492117,
        0.036029525101184845,
        0.04383246973156929,
        0.017509669065475464,
        -0.012830994091928005,
        0.05327880010008812,
        0.03357584401965141,
        0.035701021552085876,
        -0.007659628987312317,
        0.0546393021941185,
        0.0301697738468647,
        0.06672422587871552,
        -0.046022698283195496,
        0.01693817228078842,
        0.011806262657046318,
        0.003936998546123505,
        -0.011240355670452118,
        0.0011786388931795955,
        -0.029375579208135605,
        0.024837717413902283,
        0.0315120667219162,
        -0.0064033521339297295,
        -0.03034755028784275,
        0.0012185933301225305,
        -0.026156028732657433,
        -0.028364213183522224,
        -0.014785156585276127,
        -0.023612232878804207,
        -0.033567432314157486,
        0.04507479444146156,
        0.010342772118747234,
        0.017913520336151123,
        -0.009862744249403477,
        0.03357749059796333,
        -0.029133060947060585,
        0.04298035055398941,
        -0.03367117419838905,
        -0.009007562883198261,
        -0.017886117100715637,
        -0.028968125581741333,
        -0.04304014518857002,
        0.04865149408578873,
        0.06354682147502899,
        0.022926418110728264,
        0.018403328955173492,
        0.0061078001745045185,
        0.03161622956395149,
        -0.044940926134586334,
        -0.03632158041000366,
        0.012629696168005466,
        -0.017226915806531906,
        -0.00036871706834062934,
        -0.0218247901648283,
        0.012846533209085464,
        -0.03431437164545059,
        -0.0011368792038410902,
        -0.02529042772948742,
        -0.03471265733242035,
        0.06772471219301224,
        0.043360598385334015,
        -0.006714887451380491,
        -0.03617008030414581,
        -0.00457561993971467,
        0.027716154232621193,
        0.006627851165831089,
        -0.03562350571155548,
        -0.020441900938749313,
        0.052225563675165176,
        0.03767964988946915,
        -0.03862258419394493,
        -0.008651296608150005,
        -0.025007717311382294,
        -0.0044559696689248085,
        -0.008106117136776447,
        0.04543174058198929,
        0.017756246030330658,
        -0.0004085287800990045,
        0.045153312385082245,
        0.023585686460137367,
        -0.03759804368019104,
        0.03708156943321228,
        -0.006570666097104549,
        -0.053922731429338455,
        -0.04173300415277481,
        -0.008636340498924255,
        -0.01025516539812088,
        0.009113021194934845,
        -0.007424683775752783,
        -0.05943871662020683,
        -0.027214331552386284,
        -0.025032632052898407,
        -0.0311354361474514,
        -0.032364316284656525,
        0.03874112665653229,
        -0.037397511303424835,
        -0.030033526942133904,
        -0.0005279398174025118,
        -0.017971007153391838,
        0.007813438773155212,
        -0.024958541616797447,
        0.04884171858429909,
        0.06256432831287384,
        0.0389668308198452,
        -0.022728893905878067,
        -0.005482021253556013,
        0.011629974469542503,
        0.05200904607772827,
        -0.06264408677816391,
        0.010917271487414837,
        -0.005990201607346535,
        -0.02852325700223446,
        -0.021854739636182785,
        -0.00633576512336731,
        -0.036393918097019196,
        -0.018007822334766388,
        -0.025628790259361267,
        0.01983441598713398,
        0.03292937949299812,
        0.049539968371391296,
        0.022527538239955902,
        0.022807737812399864,
        0.01960667222738266,
        -0.029062561690807343,
        -0.006977609358727932,
        -0.004098282661288977,
        -0.03962192311882973,
        -0.007984012365341187,
        -0.03450188413262367,
        -0.010017778724431992,
        -0.023033035919070244,
        -0.01785822957754135,
        -0.022425636649131775,
        -0.013345390558242798,
        -0.020566415041685104,
        0.04526907205581665,
        0.026516957208514214,
        -0.03188389167189598,
        0.022206773981451988,
        -0.02115904539823532,
        -0.014001071453094482,
        0.04020985588431358,
        -0.031359679996967316,
        -0.012936879880726337,
        -0.04439079016447067,
        0.07503259927034378,
        -0.022837502881884575,
        0.01778569258749485,
        -0.029922273010015488,
        0.02062239684164524,
        -0.04601152613759041,
        0.046920545399188995,
        -0.04186465963721275,
        -0.00507931737229228,
        0.0071898228488862514,
        -0.00043745673610828817,
        0.041252732276916504,
        -0.02501652017235756,
        -0.0272219255566597,
        0.0027596887666732073,
        -0.03797705098986626,
        0.028205901384353638,
        0.006450490560382605,
        0.03546687215566635,
        -0.031317707151174545,
        0.00411933334544301,
        -0.06115598604083061,
        -0.03314933925867081,
        -0.027632862329483032,
        0.008597341366112232,
        -0.03384445235133171,
        0.021479032933712006,
        0.013721625320613384,
        0.027150187641382217,
        0.008968795649707317,
        -0.044782672077417374,
        -0.02396566979587078,
        -0.013437865301966667,
        0.04846299812197685,
        -0.041633252054452896,
        -0.03638286516070366,
        0.02431711181998253,
        -0.009860587306320667,
        0.030529063194990158,
        -0.024462617933750153,
        0.03067106194794178,
        -0.024615349248051643,
        0.023755598813295364,
        -0.06827133148908615,
        0.006729958113282919,
        0.03223295137286186,
        -0.011103970929980278,
        -0.011556900106370449,
        -0.0081463772803545,
        -0.040000151842832565,
        0.042652446776628494,
        -0.021474629640579224,
        -0.013710754923522472,
        0.039774712175130844,
        0.04511715844273567,
        0.01948435977101326,
        -0.01564585044980049,
        -0.042034123092889786,
        -0.0034682834520936012,
        0.03061145730316639,
        -0.017179889604449272,
        0.01667577587068081,
        -0.00249458197504282,
        0.019439326599240303,
        -0.028290273621678352,
        0.026383457705378532,
        -0.018050003796815872,
        0.050262171775102615,
        0.003904037643224001,
        0.042543865740299225,
        0.04246699437499046,
        -0.045440997928380966,
        -0.02162889391183853,
        -0.019635803997516632,
        -0.03623592481017113,
        0.029357686638832092,
        0.012572715058922768,
        -0.035805072635412216,
        0.040350541472435,
        -0.013477496802806854,
        -0.03315510228276253,
        -0.010893766768276691,
        -0.02883637510240078,
        -0.04656663537025452,
        -0.018608706071972847,
        -0.03364736586809158,
        -0.017188649624586105,
        -0.058947496116161346,
        -0.00045933277579024434,
        0.035392824560403824,
        -0.04189072921872139,
        0.04219147935509682,
        -0.042525243014097214,
        -0.025145413354039192,
        -0.028960449621081352,
        0.05863369256258011,
        -0.012278934940695763,
        0.013090201653540134,
        0.007323163095861673,
        -0.004392722621560097,
        -0.056028008460998535,
        0.06751296669244766,
        0.021614279597997665,
        0.03118102066218853,
        -0.03604252263903618,
        0.0036766999401152134,
        -0.04676444083452225,
        -0.0665811076760292,
        -0.027524759992957115,
        -0.05020105093717575,
        -0.02263709343969822,
        -0.04926234111189842,
        0.03158220648765564,
        0.020416997373104095,
        -0.00710417702794075,
        -0.0004680500424001366,
        0.003710005199536681,
        0.024920083582401276,
        -0.0216287299990654,
        -0.034799348562955856,
        -0.0022234476637095213,
        0.027535593137145042,
        0.02060825191438198,
        -0.05026932433247566,
        -0.017087914049625397,
        0.07333169132471085,
        -0.0007993856561370194,
        -0.05281807482242584,
        0.020267633721232414,
        0.018384922295808792,
        -0.02719900943338871,
        -0.050089672207832336,
        0.035411443561315536,
        -0.04369938001036644,
        0.012660983949899673,
        0.024846425279974937,
        0.027542347088456154,
        -0.0006186742102727294,
        -0.020103273913264275,
        -0.04720638319849968,
        0.03624683991074562,
        0.011285821907222271,
        0.020787827670574188,
        -0.05313742533326149,
        -0.020511286333203316,
        -0.013906998559832573,
        0.017560182139277458,
        0.027743887156248093,
        -0.01789954863488674,
        0.04133084416389465,
        -0.010307449847459793,
        0.019791003316640854,
        -0.002461516996845603,
        -0.07219654321670532,
        0.012353708036243916,
        -0.016594162210822105,
        -0.026361029595136642,
        -0.03974999114871025,
        0.020501768216490746,
        0.02516324073076248,
        0.009185452945530415,
        -0.0451936274766922,
        0.025183631107211113,
        -0.01344346720725298,
        -0.02420121245086193,
        0.06647174060344696,
        -0.038429901003837585,
        -0.03030518628656864,
        -0.05742467939853668,
        0.016303613781929016,
        -0.061935100704431534,
        -0.028558718040585518,
        -0.05791155993938446,
        0.0594736784696579,
        -0.0350499227643013,
        0.0326448492705822,
        0.03287243843078613,
        -0.009600327350199223,
        -0.007990570738911629,
        0.011429171077907085,
        0.06840123981237411,
        -0.01449919305741787,
        0.003352019703015685,
        -0.008939500898122787,
        -0.023839890956878662,
        -0.034037116914987564,
        -0.011406510137021542,
        -0.0002844688715413213,
        0.04662301763892174,
        -0.01913592964410782,
        -0.03770105540752411,
        -0.01773509941995144,
        -0.03511656075716019,
        0.025222737342119217,
        -0.0405912883579731,
        -0.019995108246803284,
        0.0324997715651989,
        0.024202479049563408,
        0.02965419739484787,
        0.018112221732735634,
        0.004346940200775862,
        0.03331081196665764,
        -0.00734445545822382,
        -0.019454844295978546,
        0.0393122136592865,
        -0.013142168521881104,
        -0.010857956483960152,
        -0.03467116504907608,
        -0.06299382448196411,
        0.03311753273010254,
        0.00892797764390707,
        -0.04483669996261597,
        -0.031885240226984024,
        0.024482959881424904,
        0.020453287288546562,
        -0.028597798198461533,
        0.01589725725352764,
        0.01863776333630085,
        0.030350761488080025,
        -0.004625173285603523,
        0.049845583736896515,
        0.02178032323718071,
        0.017830301076173782,
        0.005488275084644556,
        0.008970792405307293,
        0.006342022214084864,
        0.02191617153584957,
        0.0355658158659935,
        0.053803957998752594,
        0.033837590366601944,
        0.028418678790330887,
        -0.012423343025147915,
        0.023443283513188362,
        -0.02375957928597927,
        0.00958163384348154,
        0.012946703471243382,
        0.037134815007448196,
        0.04814958572387695,
        -0.010058131068944931,
        -0.012829318642616272,
        0.012537440285086632,
        -0.02397550269961357,
        -0.025352317839860916,
        0.022026238963007927,
        -0.025909217074513435,
        -0.014322910457849503,
        -0.012510688044130802,
        -0.002047247253358364,
        -0.0007060705102048814,
        -0.0016893846914172173,
        0.030694488435983658,
        -0.028184430673718452,
        0.06622341275215149,
        0.022269215434789658,
        -0.03849879279732704,
        0.011380508542060852,
        0.031608641147613525,
        0.028137193992733955,
        0.003619144205003977,
        0.010536573827266693,
        -0.045283883810043335,
        0.03769395500421524,
        0.012290563434362411,
        -0.013559863902628422,
        0.024974504485726357,
        -0.02935442328453064,
        -0.01113701518625021,
        -0.01864946447312832,
        0.02152305282652378,
        -0.037512894719839096,
        0.013396769762039185,
        -0.022216467186808586,
        -0.016414718702435493,
        -0.0017095127841457725,
        0.020582837983965874,
        0.016438307240605354,
        0.015603671781718731,
        -0.018995128571987152,
        -0.031207039952278137,
        -0.005719244014471769,
        -0.022574415430426598,
        0.011888774111866951,
        0.04043581336736679,
        0.04854315146803856,
        -0.05225171148777008,
        -0.024296322837471962,
        0.028242016211152077
      ]
print("Cosine similarity:", cosine_similarity(embedding_vector, es_vector))

Cosine similarity: 0.9999999999986178


To generate the embedding for the query inside Elasticsearch run this request in Dev Tools.

In [ ]:
POST _ml/trained_models/multilingual-e5-large-instruct/_infer
{
  "docs": [
    {
      "text_field": "Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: Radio, TV, and Film Broadcasting History"
    }
  ]
}